

```
`# This is formatted as code`
```

ขั้นตอนการทำงาน (Pipe line)
1. เตรียมข้อมูล (Dataset prepare)
2. สร้างโมเดล  (Model create)
3. สอนโมเดล  (Model training)
4. ทดสอบโมเดล (Model Evaluation)

#1. เตรียมข้อมูล (Dataset prepare)

In [1]:
!gdown --id 16xvx3v5soWmJ9f9FYbCVcLWMNZfj097j

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=16xvx3v5soWmJ9f9FYbCVcLWMNZfj097j
From (redirected): https://drive.google.com/uc?id=16xvx3v5soWmJ9f9FYbCVcLWMNZfj097j&confirm=t&uuid=5bc42405-8c28-40ab-a1ea-355b5b680e7c
To: /content/cov19.zip
100% 314M/314M [00:01<00:00, 175MB/s]


In [2]:
!unzip -qq cov19.zip

###อ่านไฟล์ในโฟลเดอร์ covid และ normal

In [3]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from os.path import isfile, join
"""
dir1 = os.listdir('train/covid/')
dir2 = os.listdir('train/normal/')
"""

"\ndir1 = os.listdir('train/covid/')\ndir2 = os.listdir('train/normal/')\n"

In [ ]:
trainpath = 'train/'
alltrainimg = []
trainImg = [trainpath+f for f in os.listdir(trainpath) if os.listdir(join(trainpath, f))]
for imagePath in (trainImg):
    for item in (os.listdir(imagePath)):
        file = os.path.join(imagePath, item)
        alltrainimg.append(file)
alltrainimg

In [ ]:
testim = cv2.imread('/content/train/covid/COVID-1.png')
testim = cv2.cvtColor(testim , cv2.COLOR_BGR2RGB)
testim=cv2.resize(testim , (400,400))
plt.imshow(testim)

In [ ]:
raw = []
label = []
for f in tqdm(alltrainimg):
  clss = f.split('/')[1]
  img = cv2.imread(f)
  img = cv2.resize(img ,(224,224))

  raw.append(img)
  if clss == 'normal':
    label.append(0)
  else:
    label.append(1)


In [ ]:
len(raw)

In [8]:
X = np.array(raw)
y = np.array(label)

In [9]:
X = X.astype('float32')
X /= 255

In [ ]:
X.shape , y.shape

In [11]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state=42)

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

#2. สร้างโมเดล (Model create)

In [13]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D


In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                                   include_top=False,
                                                   weights='imagenet')
base_model.trainable = False
num_classes = 2

modelG = Sequential([
        base_model,

        Conv2D(128, 3, activation='relu'),
        Dense(16),
        Flatten(),
        Dense(num_classes, activation='softmax')
    ])


In [ ]:
modelG.summary()

In [16]:
modelG.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics= ['accuracy'])
batch_size = 32
epochs = 1

In [ ]:
history = modelG.fit(x_train, y_train ,batch_size=batch_size, epochs=epochs ,validation_data=(x_test, y_test))

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,4))
plt.subplot(121),
plt.title('model accuracy');plt.ylabel('accuracy');plt.xlabel('epoch');plt.legend(['train', 'validation'], loc='upper left')
plt.plot(history.history['accuracy']);plt.plot(history.history['val_accuracy'])

plt.subplot(122)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.plot(history.history['loss']);plt.plot(history.history['val_loss'])

In [ ]:
from os import listdir
width = 224
testpath = 'test/'
testImg = [testpath+f for f in listdir(testpath)]
rimg = []
y_true = []
y_predict = []
for imagePath in (testImg):
    for i,item in enumerate(listdir(imagePath)):
        file = join(imagePath, item)
        if file[-1] =='g':
          #print(file.split('/'))
          if file.split('/')[1] == 'normal':
            y_true.append(0)
          else:
            y_true.append(1)
          imgori = cv2.imread(file)
          imgori = cv2.cvtColor(imgori , cv2.COLOR_BGR2RGB)
          img = cv2.resize(imgori ,(width,width))
          rimg = np.array(img)
          rimg = rimg.astype('float32')
          rimg /= 255
          rimg = np.reshape(rimg ,(1,224,224,3))
          predict = modelG.predict(rimg)
          label = ['normal','covid']
          print(f"Pred:{np.argmax(predict)}")
          result = label[np.argmax(predict)]
          y_predict.append(np.argmax(predict))
          plt.title(item)
          cv2.putText(imgori, str(result), (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,0,0), 2 )

          plt.imshow(imgori)
          plt.show()

In [ ]:

#y_true = [np.argmax(i) for i in y_true]
y_true


In [ ]:
y_predict

In [ ]:
correct = 0
for i in range(len(y_true)):
  if y_true[i] == y_predict[i]:
    correct += 1

print(f"Accuracy: {(correct/len(y_true))*100}%")

In [ ]:
from sklearn.metrics import classification_report
clss = ['normal','covid']
classreport = classification_report(y_true ,y_predict, target_names=clss )
print(classreport)

In [ ]:
import seaborn as sb
from sklearn.metrics import confusion_matrix

cf = confusion_matrix(y_true , y_predict)
plt = sb.heatmap(cf, annot=True)
plt.set_title('Confusion Matrix');
plt.set_xlabel('Predicted Values')
plt.set_ylabel('Actual Values ')

plt.xaxis.set_ticklabels(clss)
plt.yaxis.set_ticklabels(clss)